In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [4]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

In [5]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = os.path.join(AWS_folder,'dataDownloaded')
src_folder

out_folder = os.path.join(AWS_folder,'ToUploadLabel')
out_folder

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\ToUploadLabel'

### 골프장 촬영일자 별로 최근버젼 저장

In [6]:

courseid = "MGC004"
date = "20230902"


response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  FilterExpression=Attr('date').eq(date))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid), 
     ExclusiveStartKey=response['LastEvaluatedKey'] ,FilterExpression=Attr('date').eq(date) )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [7]:
len(data)

15674

In [8]:
from decimal import Decimal

#Does quasi the same things as json.loads from here: https://pypi.org/project/dynamodb-json/
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return float(obj)
        return json.JSONEncoder.default(self, obj)

file_to_save = os.path.join(b_folder ,'{}.json'.format(courseid+'_'+date))
print(file_to_save)
with open(file_to_save, "w", encoding="utf-8") as file:
  file.write(json.dumps(data, cls=JSONEncoder, ensure_ascii=False))

c:\Users\yc463\Documents\JamesLocal\DS\Drone\dsmapv2\src\AWS\BACKUPS\MGC004_20230902.json


### Delete DB with target course & date

In [9]:
with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(data):
    print(index_, end='\r')
    batch.delete_item(
            Key = {
                "metadata": row_["metadata"],
                "id": row_["id"]
            }
        )

###Dummy Routine for FUture

In [27]:
df = pd.DataFrame(data)
df['newlabel'] = df.json.map(lambda x: x['label'])

In [28]:
def replace_label(x, y):
  x.update({'label':y})
  return x

In [29]:

df.json = df.apply(lambda x: replace_label(x['json'],x['newlabel']), axis=1)

In [30]:
df.iloc[0,:]['json']

{'originalFileGrp': ['original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1037-1046_건조,패취류,썸머패취,켄터키,티,답압,장비피해\\DJI_20230823103730_0001_MS_G.TIF',
  'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1037-1046_건조,패취류,썸머패취,켄터키,티,답압,장비피해\\DJI_20230823103730_0001_MS_NIR.TIF',
  'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1037-1046_건조,패취류,썸머패취,켄터키,티,답압,장비피해\\DJI_20230823103730_0001_MS_R.TIF',
  'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1037-1046_건조,패취류,썸머패취,켄터키,티,답압,장비피해\\DJI_20230823103730_0001_MS_RE.TIF'],
 'annotation': [],
 'labelBy': 'ftp_dsg',
 'GeoTagInfo': {'gps_altitude': Decimal('130.632'),
  'datetime_original': '2023:08:23 10:37:30',
  'coords': [Decimal('128.65041494444446'), Decimal('35.65462727777778')],
  'gps_altitude_ref': Decimal('0')},
 'id': '20230823103730_1286504149444_356546272778_130632000',

In [31]:
file2save = os.path.join(out_folder, courseid + '_' + date+'.json')
with open(file2save, 'w', encoding="utf-8") as f:
  f.write(df.to_json(orient='records', force_ascii=False))

In [32]:
json.loads(df.to_json(orient='records', force_ascii=False))

[{'metadata': 'MGC002202308231_밸리6H건조,패취류,썸머패취,켄터키,티,답압,장비피해',
  'date': '20230823',
  'area': '1_밸리6H',
  'courseid': 'MGC002',
  'json': {'originalFileGrp': ['original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1037-1046_건조,패취류,썸머패취,켄터키,티,답압,장비피해\\DJI_20230823103730_0001_MS_G.TIF',
    'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1037-1046_건조,패취류,썸머패취,켄터키,티,답압,장비피해\\DJI_20230823103730_0001_MS_NIR.TIF',
    'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1037-1046_건조,패취류,썸머패취,켄터키,티,답압,장비피해\\DJI_20230823103730_0001_MS_R.TIF',
    'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1037-1046_건조,패취류,썸머패취,켄터키,티,답압,장비피해\\DJI_20230823103730_0001_MS_RE.TIF'],
   'annotation': [],
   'labelBy': 'ftp_dsg',
   'GeoTagInfo': {'gps_altitude': 130.632,
    'datetime_original': '2023:08:23 10:37:30',
    'coords': [128.6504149444, 